# parameters : [0.4, 2, 100, 2, 0.5, 0.5, 1, 4]
# accuracy : 0.6875
# spes     : 0.6842006033182504
# sens     : 0.735
# auc      : 0.7096003016591252
# weights  : [0.15, 0.4, 0.1, 0.05, 0.05, 0.25]

In [3]:
# Importing the libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import sys
import numpy
numpy.set_printoptions(threshold=sys.maxsize)

# Importing the dataset
dataset = pd.read_csv('CVD dataset-1-3-7-8-10-11-13-15.csv')
data=dataset.iloc[:, :].values
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, 8].values
# X = dataset.iloc[:, [ 0, 1, 2, 5, 10, 11, 13, 15]].values

rows = len(data)    # 3 rows in your example
cols = len(data[0])
print(rows)
print(cols)

560
9


In [4]:
dataNoCvd = data[~(data[:,8] > 0.0)]
dataCvd = data[~(data[:,8] < 1.0)]
rowsNoCvd = len(dataNoCvd)    
rowsCvd = len(dataCvd)
print(rowsNoCvd)
print(rowsCvd)
# print(dataNoCvd)
# print(dataCvd)

519
41


In [5]:
def find_NoCvd_limitsArray(i,DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue):
    if i>0 :
        DownNoCvd = UpNoCvd
        UpNoCvd= fold_NoCvd_total+UpNoCvd
    if i < fold_NoCvd_residue :
        UpNoCvd+= 1
#     print("NoCvd limits, Down :",DownNoCvd,"Up ",UpNoCvd)
    return   DownNoCvd, UpNoCvd   
 
def find_Cvd_limitsArray(i,DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv):
    if i>0 :
        DownCvd = UpCvd
        UpCvd= fold_Cvd_total+UpCvd
    if i >= cv - fold_Cvd_residue :
         UpCvd+= 1     
#     print("Cvd limits, Down :",DownCvd,"Up ",UpCvd)
    return   DownCvd, UpCvd  

def find_testSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1=dataNoCvd[DownNoCvd:UpNoCvd,:]
    temp2=dataCvd[DownCvd:UpCvd,:]
    temp3=np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

def find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd):
    temp1 = np.delete(dataNoCvd, slice(DownNoCvd, UpNoCvd), axis=0)
    temp2 = np.delete(dataCvd, slice(DownCvd, UpCvd), axis=0)
    temp3 = np.concatenate((temp1, temp2))
#     print(len(temp1),"  ",len(temp2),"  ",len(temp3))
    return temp1, temp2, temp3

In [6]:
def create_train_test_sets(cv,dataNoCvd,dataCvd):
    test_total = []
    train_total = []
    train_total_Cvd = []
    train_total_NoCvd = []
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)

    fold_Cvd_total = rowsCvd//cv
    fold_Cvd_residue= rowsCvd%cv
    print("fold_Cvd_total  :",fold_Cvd_total," fold_Cvd_residue  :",fold_Cvd_residue)

    fold_NoCvd_total = rowsNoCvd//cv
    fold_NoCvd_residue= rowsNoCvd%cv
    print("fold_NoCvd_total:",fold_NoCvd_total,"fold_NoCvd_residue:",fold_NoCvd_residue)

    DownNoCvd=0
    DownCvd=0
    UpNoCvd = fold_NoCvd_total
    UpCvd = fold_Cvd_total

    for i in range(cv):
        X_test = []
        y_test = []
        X_train = []
        y_train = []
#         print("============",i,"==============")
        DownNoCvd, UpNoCvd = find_NoCvd_limitsArray(i, DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue)
        DownCvd, UpCvd = find_Cvd_limitsArray(i, DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv)

        testSubset_NoCvd ,testSubset_Cvd ,testSubset_total = find_testSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
        trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd)
     
    #creating X_train, y_train, X_test, y_test
        X_test.append(np.delete(testSubset_total, 8, axis=1))
        y_test_temp = np.delete(testSubset_total, slice(0, 8), axis=1)
        y_test.append(np.reshape(y_test_temp, len(y_test_temp)))
        X_test_temp = np.array(X_test)
        X_test = X_test_temp[0]
        y_test_temp = np.array(y_test)
        y_test = y_test_temp[0]
    
        X_train.append(np.delete(trainSubset_total, 8, axis=1))
        y_train_temp = np.delete(trainSubset_total, slice(0, 8), axis=1)
        y_train.append(np.reshape(y_train_temp, len(y_train_temp)))
        X_train_temp = np.array(X_train)
        X_train = X_train_temp[0]
        y_train_temp = np.array(y_train)
        y_train = y_train_temp[0]
        
    #add every subset in a list so we can handle thm later 
        test_total.append([X_test,y_test])
        train_total.append([X_train,y_train]) 
        train_total_NoCvd.append(trainSubset_NoCvd)
        train_total_Cvd.append(trainSubset_Cvd)
    return train_total, test_total, train_total_NoCvd, train_total_Cvd

In [7]:
train_total, test_total, train_total_NoCvd, train_total_Cvd = create_train_test_sets(10,dataNoCvd,dataCvd)

fold_Cvd_total  : 4  fold_Cvd_residue  : 1
fold_NoCvd_total: 51 fold_NoCvd_residue: 9


In [8]:
# Chech ratio in each train and test set
def find_ratio(index_list):
    one = 0
    lenght=len(index_list[0])
    for i in range(lenght):
#         print(index_list[1])
        if index_list[1][i] == 1.0 :
            one+= 1
    ratio = one/lenght
    return ratio

for i in range(10):
    print(len(train_total[i][0]))
    ratio = find_ratio(train_total[i])
    print("ratio in train set:", ratio)
    ratio = find_ratio(test_total[i])
    print("ratio in test set:", ratio)

504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07341269841269842
ratio in test set: 0.07142857142857142
504
ratio in train set: 0.07142857142857142
ratio in test set: 0.08928571428571429


In [9]:
from xgboost import XGBClassifier


def my_fit(ratio,dataNoCvd,dataCvd,params):   
    Subarray=[]
    models = []
    # yarray
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)
#     print("Size of NoCvd cases in train set:",rowsNoCvd)
#     print("Size of Cvd cases in train set  :",rowsCvd)


    numOfSubsamples = rowsNoCvd//(rowsCvd*ratio)
    numOfSubsamples= int(numOfSubsamples)
    SubNoCvd = rowsNoCvd//numOfSubsamples
    residue = rowsNoCvd- SubNoCvd*numOfSubsamples


    Up = 0


    for i in range(numOfSubsamples):
        
        classifier = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=params[5],
                  colsample_bynode=1, colsample_bytree=0.75, gamma=params[4],
                  learning_rate=params[0], max_delta_step=0, max_depth=params[1],
                  min_child_weight=params[3], missing=None, n_estimators=params[2], n_jobs=1,
                  nthread=None, objective='binary:logistic', random_state=0,
                  reg_alpha=0.1, reg_lambda=params[6], scale_pos_weight=params[7], seed=None,
                  silent=None, subsample=0.8, verbosity=1)
    #     print(i)
    #Creating the training set for each model
        Down = Up
        Up= Up + SubNoCvd
        if i < residue :
            Up+= 1
        
#         print(Down)
#         print(Up)
        
        Sub1=dataNoCvd[Down:Up,:]
        Sub2=np.concatenate((Sub1, dataCvd))
        Subarray.append(Sub2)
        
    #Dividing to X and y of the previous traing set    
        X=np.delete(Subarray[i], 8, axis=1)
        y=np.delete(Subarray[i], slice(0, 8), axis=1)
        y=np.reshape(y, len(y))
        
        classifier.fit( X, y)
        models.append(classifier)
    return models     

In [10]:


def predict_weighted_average(models,X,weights):
    y_pred = []
    y_pred_models = []
#     weights = [0.16, 0.175, 0.2, 0.147, 0.148, 0.16]
#     weights = [0.15, 0.4, 0.2, 0.05, 0.05, 0.15]
#   weights = [0.16, 0.173, 0.2, 0.147, 0.16, 0.16]
    models_size = len(models)
    X_size = len(X)
            
    for i in range(models_size):
        y_pred_models.append(models[i].predict_proba(X)[:,1]) 
        
    for j in range(X_size):
        sum = 0
        for i in range(models_size):
            
            sum = sum + y_pred_models[i][j]*weights[i]
        avr = sum
#         /models_size
        if avr>= 0.5 :
            y_pred.append(1.)
        else:
            y_pred.append(0.)
    return y_pred  

In [18]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from sklearn import metrics

def my_cross_val( cv, train_total, test_total, train_total_NoCvd, train_total_Cvd,params, weights):
    accuracy = []
    specificity = []
    sensitivity = []
    auc = []
    
    # print(X)
    for i in range(cv):
#         X_train = train_total[i][0]
#         y_train = train_total[i][1]
        X_test = test_total[i][0]
        y_test = test_total[i][1]
        


        models = my_fit( 2, train_total_NoCvd[i], train_total_Cvd[i],params)


        y_pred = predict_weighted_average(models,X_test,weights)

        cm = confusion_matrix(y_test, y_pred)
    #     print(cm)

        total1=sum(sum(cm))
        #####from confusion matrix calculate accuracy
        accuracy1=(cm[0,0]+cm[1,1])/total1
    #   print ('Accuracy : ', accuracy1)

        specificity1 = cm[0,0]/(cm[0,0]+cm[0,1])
    #   print('Specificity : ', specificity1 )

        sensitivity1 = cm[1,1]/(cm[1,0]+cm[1,1])
    #   print('Sensitivity : ', sensitivity1)
        
#         y = np.array(y_test)
#         pred = np.array(y_pred)
#         fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=2)
#         auc1 = metrics.auc(fpr, tpr)
        
        auc1 = roc_auc_score(y_test, y_pred)

    
        accuracy.append(accuracy1)
        specificity.append(specificity1)
        sensitivity.append(sensitivity1) 
        auc.append(auc1)
    return accuracy, specificity, sensitivity, auc

In [12]:
params = {
                    "learning_rate"    :[0.4],  #0.01-0.2 Makes the model more robust by shrinking the weights on each step
                    "max_depth"        :[2],  #3-10 control over-fitting as higher depth will allow model to learn relations very specific to a particular sample
                    "n_estimators"     :[100],
                    "min_child_weight" :[2],         #0.5-1 small values might lead to under-fitting
                    "gamma"            :[0.5],            #Makes the algorithm conservative --> No overfitting
                    "colsample_bytree" :[0.5], #0.5-1
                     "reg_lambda"      :[1], #  it should be explored to reduce overfitting.
                    "scale_pos_weight" :[4]
        }

# parameters : [0.4, 2, 100, 2, 0.5, 0.5, 1, 4]
# accuracy : 0.6875
# spes     : 0.6842006033182504
# sens     : 0.735
# auc      : 0.7096003016591252
# weights  : [0.15, 0.4, 0.1, 0.05, 0.05, 0.25]
    
#  [[2.8236261372771025, 15], [2.5580499288255822, 13], [2.556403643206909, 1], [2.3482621568517854, 11], 
#   [2.310497939649767, 10], [2.302223741666667, 7], [2.25283855005, 8], [2.2489727393473724, 3]]
# 1 3 7 8 10 11 13 15

In [19]:
comb = compute_comb(params)
print(comb[0])
# weights  = [0.15, 0.2, 0.15, 0.25, 0.15, 0.1]
# weights = [0.28, 0.15, 0.1, 0.1, 0.17, 0.2]
weights = [0.15, 0.4, 0.1, 0.05, 0.05, 0.25]
accuracy, specificity, sensitivity, auc = my_cross_val( 10, train_total, test_total, train_total_NoCvd, train_total_Cvd,comb[0],weights)

[0.4, 2, 100, 2, 0.5, 0.5, 1, 4]


In [21]:
from statistics import mean
print(mean(accuracy))
print(mean(specificity))
print(mean(sensitivity))
print(mean(auc))

0.6464285714285715
0.6552790346907994
0.54
0.5976395173453997


In [14]:
# https://www.geeksforgeeks.org/python-all-possible-permutations-of-n-lists/

def compute_comb(params):
    
    # initializing lists 
    list1 = params.get("learning_rate")
    list2 = params.get("max_depth")
    list3 = params.get("n_estimators") 
    list4 = params.get("min_child_weight")
    list5 = params.get("gamma")    
    list6 = params.get("colsample_bytree")
    list7 = params.get("reg_lambda") 
    list8 = params.get("scale_pos_weight")

#     # printing lists  
#     print ("The original lists are : " + str(list1) +
#                                    " " + str(list2) + 
#                                    " " + str(list3) + 
#                                    " " + str(list4) + 
#                                    " " + str(list5) + 
#                                    " " + str(list6) +
#                                    " " + str(list7) +
#                                    " " + str(list8)) 

    # using list comprehension  
    # to compute all pmax_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)max_sens ,max_params, max_spes, max_acc, max_auc, best_params = gridsearch(params)ossible permutations 
    res = [[i, j, k, l, m, n, o, p] for i in list1  
                                    for j in list2 
                                    for k in list3 
                                    for l in list4 
                                    for m in list5 
                                    for n in list6 
                                    for o in list7
                                    for p in list8] 

    # printing result 
#     print ("All possible permutations are : " +  str(res))
    return res

In [15]:
class style:
    BOLD = '\033[1m'
    END = '\033[0m'

In [16]:
# https://towardsdatascience.com/train-test-split-and-cross-validation-in-python-80b61beca4b6

from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from imxgboost.imbalance_xgb import imbalance_xgboost as imb_xgb
from sklearn.model_selection import StratifiedKFold # import KFold
from statistics import mean

def gridsearch(params):
    max_sens = 0
    max_spes = 0
    max_acc = 0
    best_params = []
    max_params = []
    max_auc = 0
    combs = compute_comb(params)
    print(len(combs))
    for j in range(len(combs)):
        print(j)
        accuracy, specificity, sensitivity, auc = my_cross_val( 10, train_total, test_total, train_total_NoCvd, train_total_Cvd,combs[j])
        if mean(sensitivity)> 0.5 and mean(specificity)>0.6 and mean(accuracy)>0.6 and mean(auc)>0.5:
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
            print(combs[j])
            print (style.BOLD + 'accuracy    ' + style.END, mean(accuracy))
            print (style.BOLD + 'specificity ' + style.END, mean(specificity))
            print (style.BOLD + 'sensitivity ' + style.END, mean(sensitivity))
            print (style.BOLD + 'auc         ' + style.END, mean(auc))
            print(style.BOLD + "----------------------------------------------------------------"+ style.END)
        else:
            print(combs[j])
            print("accuracy    ",mean(accuracy))
            print("specificity ",mean(specificity))
            print("sensitivity ",mean(sensitivity))
            print("auc         ",mean(auc))
        best_params.append([combs[j],mean(accuracy),mean(specificity),mean(sensitivity),mean(auc)])
        if mean(auc)> max_auc:
            max_sens = mean(sensitivity)
            max_spes = mean(specificity)
            max_acc = mean(accuracy)
            max_auc = mean(auc)
            max_params = combs[j]
    return max_sens, max_params, max_spes, max_acc, max_auc, best_params